In [3]:
# Final -- July 28 2024

"""
Install the Google AI Python SDK

$ pip install google-generativeai

See the getting started guide for more information:
https://ai.google.dev/gemini-api/docs/get-started/python
"""

import os

import google.generativeai as genai

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
# See https://ai.google.dev/api/python/google/generativeai/GenerativeModel
generation_config = {
    "temperature": 0.6,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
    model_name="gemini-1.5-pro",
    generation_config=generation_config,
    # safety_settings = Adjust safety settings
    # See https://ai.google.dev/gemini-api/docs/safety-settings
    system_instruction="""
        Imagine you're a specialist in [Future desired job / industry post-grad here]. A job the current student you're facing wants to have. To make this math problem more engaging, rephrase it in the context of the student's desired career: [Future desired job/industry post-grad here].

        Ensure the numerical values and core questions remain unchanged; only modify the context to make it more intriguing for me. Find the answer internally using Code Execution first. Instead of directly providing the answer, keep asking me questions until I give you my input. A call and response scenario.

        Do it in a step-by-step manner. Really breakdown the proble and ask one question at a time instead of giving me the whole formula or spoonfeeding me the given variables.

        If I'm confused and you need to provide a formula, make sure to ask me what to do with it. Don't just give it to me. Make me think and engage with the problem. Let me do the plugging of values.

        Just make sure to run code execution to verify my answers.

        End your first response by starting with asking me what to do in the first step and providing the formula and the given.

        Keep responses humanely short. 2-3 sentences per response.
        """,
    tools="code_execution",
)

chat_session = model.start_chat(history=[])

interest = input("What are you interested in? ")
problem = input(
    "What is the math problem? If you don't have one, we will give you a sample. Just type 'skip' "
)
level = input(
    "What is kind of math do you want to practice solving? College-level or everyday math? "
)


if problem != "skip":
    prompt_hasProblem = f"I'm interested in {interest}. The math I have to solve is: {problem}. Keep in mind my proficiency level or the type I want to practice on is {level} mathematics."
    response = chat_session.send_message(prompt_hasProblem)

prompt_noProblem = f"I'm interested in {interest}. Please create a simple sample math problem. Note that my proficiency level or the type I want to practice on is {level} mathematics."
response = chat_session.send_message(prompt_noProblem)

print(response.text)

while True:
    user_input = input("You: ")
    response = chat_session.send_message(user_input)
    print(f"AI: {response.text}")
    if "Goodbye" in response.text:
        break


I'd love to help you out with this! To make it more engaging, let's pretend we're both [Future desired job / industry post-grad here]. 

Let's dive into this problem. First, we need to figure out [Rephrase the first step of the calculation in the context of the job].  

Given [Quantity 1] and [Quantity 2], what would you do first?

To calculate this, we can use the formula:

[Insert relevant formula]

Where:

* [Variable 1] = [Quantity 1]
* [Variable 2] = [Quantity 2] 



# Tryhard -v

In [1]:
import os
import google.generativeai as genai


genai.configure(api_key="")

code_model = genai.GenerativeModel(model_name="gemini-1.5-pro", tools="code_execution")

# Global debug variable
debug = True


def debug_print(message):
    if debug:
        print(f"[DEBUG] {message}")


# Configure Gemini API
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
generation_config = {
    "temperature": 0.9,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
}

model = genai.GenerativeModel(
    model_name="gemini-1.5-pro",
    generation_config=generation_config,
    tools=["code_execution"],
    system_instruction="""Imagine you're a specialist in [Future desired job / industry post-grad here]. To\nmake this problem more engaging, rephrase it in the context of [Future desired\njob/industry post-grad here]. Ensure the numerical values and core questions\nremain unchanged; only modify the context to make it more intriguing for me.\n\nFind the answer internally using Code Execution first. Instead of directly providing the answer, keep asking me questions until I give you my input. Do it in a step-by-step manner.\nEnd your first response by starting with the first step and providing the formula\nand the given.\n\nThe Math Problem\n“[Paste math problem directly – problems not properly formatted due to the\nconversion from Latex work fine as they can be understood by Gemini. Otherwise, print the problem and confirm with the user if that's what the original question was.]""",
)

chat_session = model.start_chat(history=[])


def preprocess_input(user_input):
    debug_print("Entering preprocess_input")
    prompt = """
    # High-level instructions:
    1. Remove filler words and clean up the input.
    2. Maintain the original idea without deviating from the main point.
    3. If it's a math question, make it clear for future processing without rephrasing.
    4. If it's not a math question, clarify the user's intent.

    # Example input/output:
    Input: "Um, can you like help me solve this Python math thing? It's about calculating factorial or something."
    Output: "Help solve a Python math problem about calculating factorial"

    # Now, preprocess the following input:
    {user_input}
    """
    response = chat_session.send_message(prompt.format(user_input=user_input))
    cleaned_input = response.text
    debug_print(f"Preprocessed input: {cleaned_input}")
    return cleaned_input


def classify_input(cleaned_input):
    debug_print("Entering classify_input")
    prompt = """
    # High-level instructions:
    1. Analyze the given input and classify it into one of the following categories:
       - interest_profiling
       - question_analysis
       - socratic_dialogue
       - answer_verification
       - learning_reinforcement
    2. Base your classification on the overall intent of the input, not just keywords.
    3. Return only the category name, nothing else.

    # Example input/output:
    Input: "What are some math concepts used in Python programming?"
    Output: interest_profiling

    Input: "Can you help me understand how to calculate factorial in Python?"
    Output: question_analysis

    # Now, classify the following input:
    {cleaned_input}
    """
    response = chat_session.send_message(prompt.format(cleaned_input=cleaned_input))
    category = response.text.strip().lower()
    debug_print(f"Classified as: {category}")
    return category


def interest_profiling_agent(user_input):
    debug_print("Entering interest_profiling_agent")
    # 3. Prepare a brief interest profile that can be used in future math and Python discussions.
    prompt = """
    # High-level instructions:
    1. Determine their interests in Python and mathematics.
    2. Focus on how their interests relate to Python programming and mathematical concepts.
    3. Return only one phrase, which is their area of interest but nuanced.
    4. Return the interest profile in a concise format. Very short and to the point.
    5. Don't ask a question to end as the user will start exploring the math problem.

    # Example input/output:
    Input: "I love data analysis and machine learning."
    Output: "Interest Profile: Data Analysis, Machine Learning. Math Connections: Statistics, Linear Algebra, Calculus. Python Libraries: NumPy, Pandas, Scikit-learn."

    # Now, create an interest profile based on the user's input:
    {user_input}
    """
    response = chat_session.send_message(prompt.format(user_input=user_input))
    return response.text


def question_analysis_agent(user_input):
    debug_print("Entering question_analysis_agent")
    prompt = """
    # High-level instructions:
    1. Analyze the given Python math question or problem.
    2. Identify the key mathematical and Python programming concepts involved.
    3. Suggest a step-by-step approach to solving the problem using Python.
    4. Do not solve the problem, but prepare it for Socratic dialogue.

    # Example input/output:
    Input: "How do I calculate factorial in Python?"
    Output: "
    Problem: Calculate factorial in Python
    Key Concepts: Recursion, Loops, Integer arithmetic
    Python Concepts: Functions, Conditional statements
    Suggested Approach:
    1. Understand the mathematical definition of factorial
    2. Discuss possible implementations (recursive vs. iterative)
    3. Plan the function structure
    4. Implement the chosen approach
    5. Test the function with various inputs
    "

    # Now, analyze the following Python math problem:
    {user_input}
    """
    response = chat_session.send_message(prompt.format(user_input=user_input))
    return response.text


def socratic_dialogue_agent(user_input, question_analysis):
    debug_print("Entering socratic_dialogue_agent")
    prompt = """
    # High-level instructions:
    1. Use the question analysis provided to guide a Socratic dialogue about Python and math.
    2. Ask open-ended questions that encourage critical thinking about both mathematical concepts and Python implementation.
    3. Provide hints and scaffolding when necessary, but avoid giving direct answers or code solutions.
    4. Use the user's interests (if available) to make connections to the Python and math concepts.
    5. Focus on one step of the problem-solving process at a time.

    # Example input/output:
    Input:
    User: "I'm not sure how to start calculating factorial in Python"
    Question Analysis: "
    Problem: Calculate factorial in Python
    Key Concepts: Recursion, Loops, Integer arithmetic
    Python Concepts: Functions, Conditional statements
    Suggested Approach:
    1. Understand the mathematical definition of factorial
    2. Discuss possible implementations (recursive vs. iterative)
    3. Plan the function structure
    4. Implement the chosen approach
    5. Test the function with various inputs
    "
    Interest Profile: Data Analysis, Machine Learning

    Output: "Let's approach this step-by-step, relating it to data analysis where we can. In data analysis, you often need to calculate probabilities, which sometimes involve factorials. First, let's think about the mathematical definition of factorial. Can you tell me what n! (n factorial) means mathematically?"

    # Now, engage in a Socratic dialogue based on the following input:
    User Input: {user_input}
    Question Analysis: {question_analysis}
    """
    response = chat_session.send_message(
        prompt.format(user_input=user_input, question_analysis=question_analysis)
    )
    return response.text


def answer_verification_agent(user_input, question_analysis):
    debug_print("Entering answer_verification_agent")
    prompt = """
    # High-level instructions:
    1. Analyze the user's proposed Python solution or answer.
    2. Use Python code to verify the correctness of the answer.
    3. Provide feedback on the correctness of the solution.
    4. If the answer is incorrect, offer constructive feedback and hints for improvement.
    5. If the answer is correct, congratulate the user and explain why their approach worked.

    # Example input/output:
    Input:
    User: "I think this Python function calculates factorial: def factorial(n): return 1 if n == 0 else n * factorial(n-1)"
    Question Analysis: "
    Problem: Calculate factorial in Python
    Key Concepts: Recursion, Integer arithmetic
    Python Concepts: Functions, Conditional statements
    "

    Output:
    ```python
    def factorial(n):
        return 1 if n == 0 else n * factorial(n-1)

    # Test the function
    test_cases = [0, 1, 5, 10]
    for n in test_cases:
        result = factorial(n)
        print(f"factorial({n}) = {result}")
    ```

    The Python code shows that:
    factorial(0) = 1
    factorial(1) = 1
    factorial(5) = 120
    factorial(10) = 3628800

    Great job! Your recursive factorial function is correct. It correctly handles the base case (n = 0) and recursively calculates larger factorials. The function successfully passes all test cases.

    Here's why your approach works:
    1. Base case: When n is 0, it returns 1, which is the correct value for 0!
    2. Recursive case: For n > 0, it multiplies n by the factorial of (n-1), which is the mathematical definition of factorial.
    3. The recursion continues until it reaches the base case, ensuring all numbers from n down to 1 are multiplied together.

    Well done on implementing a concise and correct recursive solution!

    # Now, verify the following Python solution:
    User Input: {user_input}
    Question Analysis: {question_analysis}
    """
    response = chat_session.send_message(
        prompt.format(user_input=user_input, question_analysis=question_analysis)
    )
    return response.text


def learning_reinforcement_agent(user_input, question_analysis):
    debug_print("Entering learning_reinforcement_agent")
    prompt = """
    # High-level instructions:
    1. Summarize the key Python and math concepts practiced in the recent problem-solving session.
    2. Highlight connections between the math concepts, Python implementation, and real-world applications, especially relating to the user's interests if available.
    3. Suggest related Python problems or topics for further practice.
    4. Provide encouragement and motivation for continued learning in both Python and mathematics.

    # Example input/output:
    Input:
    User: "Can you summarize what we learned from implementing factorial in Python?"
    Question Analysis: "
    Problem: Calculate factorial in Python
    Key Concepts: Recursion, Integer arithmetic
    Python Concepts: Functions, Conditional statements
    "
    Interest Profile: Data Analysis, Machine Learning

    Output: "Great question! Let's recap what we've learned:

    1. Mathematical Concept: We explored factorial, a fundamental concept in combinatorics and probability.
    2. Recursive Thinking: We implemented factorial using recursion, a powerful technique in both mathematics and programming.
    3. Python Implementation:
       - We defined a function using 'def'
       - We used conditional statements for the base and recursive cases
       - We applied recursive function calls

    Real-world connections:
    - In data analysis, factorials are used in calculating permutations and combinations, which are essential in probability and statistics.
    - In machine learning, understanding recursion can help in implementing and understanding algorithms like decision trees or in processing hierarchical data structures.

    Python and Math Synergy:
    This problem beautifully demonstrates how mathematical concepts can be translated into code, and how programming can help us understand mathematical ideas more deeply.

    For further practice, try these related Python problems:
    1. Implement an iterative (non-recursive) version of factorial.
    2. Write a function to calculate combinations (nCr) using your factorial function.
    3. Use your factorial function to approximate the mathematical constant e.

    Remember, each Python problem you solve strengthens both your programming skills and mathematical understanding, which are crucial in data analysis and machine learning. Keep up the great work!"

    # Now, provide a learning summary based on the following input:
    User Input: {user_input}
    Question Analysis: {question_analysis}
    """
    response = chat_session.send_message(
        prompt.format(user_input=user_input, question_analysis=question_analysis)
    )
    return response.text


def chatbot(user_input):
    debug_print("Chatbot responding...")

    cleaned_input = preprocess_input(user_input)
    category = classify_input(cleaned_input)

    if category == "question_analysis":
        question_analysis = question_analysis_agent(cleaned_input)
        response = socratic_dialogue_agent(cleaned_input, question_analysis)
        chat_session.history.append({"role": "assistant", "parts": [response]})
    elif category == "socratic_dialogue":
        last_question_analysis = next(
            (
                msg
                for msg in reversed(chat_session.history)
                if "Question Analysis:" in msg["parts"][0]
            ),
            None,
        )
        if last_question_analysis:
            question_analysis = (
                last_question_analysis["parts"][0]
                .split("Question Analysis:", 1)[1]
                .strip()
            )
            response = socratic_dialogue_agent(cleaned_input, question_analysis)
        else:
            response = "I'm sorry, but I don't have the context of the previous question. Could you please restate the Python math problem you're working on?"
        chat_session.history.append({"role": "assistant", "parts": [response]})
    elif category == "answer_verification":
        last_question_analysis = next(
            (
                msg
                for msg in reversed(chat_session.history)
                if "Question Analysis:" in msg["parts"][0]
            ),
            None,
        )
        if last_question_analysis:
            question_analysis = (
                last_question_analysis["parts"][0]
                .split("Question Analysis:", 1)[1]
                .strip()
            )
            response = answer_verification_agent(cleaned_input, question_analysis)
        else:
            response = "I'm sorry, but I don't have the context of the previous question. Could you please restate the Python math problem and your proposed solution?"
        chat_session.history.append({"role": "assistant", "parts": [response]})
    elif category == "learning_reinforcement":
        last_question_analysis = next(
            (
                msg
                for msg in reversed(chat_session.history)
                if "Question Analysis:" in msg["parts"][0]
            ),
            None,
        )
        if last_question_analysis:
            question_analysis = (
                last_question_analysis["parts"][0]
                .split("Question Analysis:", 1)[1]
                .strip()
            )
            response = learning_reinforcement_agent(cleaned_input, question_analysis)
        else:
            response = "I'd be happy to summarize what we've learned, but I don't have the context of our previous discussion. Could you please remind me which Python math concept or problem we were working on?"
        chat_session.history.append({"role": "assistant", "parts": [response]})
    else:
        response = "I'm not sure how to respond to that. Could you please ask about a specific Python math problem or concept you'd like help with?"

    debug_print(f"Chatbot response: {response}")
    return response

In [2]:
# Test the chatbot
if __name__ == "__main__":
    debug_print("Entering chatbot — tell me your interests!")
    user_input = input()

    interest_identified = interest_profiling_agent(user_input)

    debug_print(f"Interest Identified: {interest_identified}")
    debug_print("Paste your math question.")

    debug_print(f"Gathered User Interest: {interest_identified}")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Thank you for using the SAT Math Preparation Chatbot. Goodbye!")
            break

        response = chatbot(user_input)
        print(f"Chatbot: {response}")

[DEBUG] Entering chatbot — tell me your interests!
[DEBUG] Entering interest_profiling_agent
[DEBUG] Interest Identified: Interest Profile: Precision Agriculture. Math Connections: Statistics, Modeling. Python Libraries: NumPy, Pandas, GeoPandas. 

[DEBUG] Paste your math question.
[DEBUG] Gathered User Interest: Interest Profile: Precision Agriculture. Math Connections: Statistics, Modeling. Python Libraries: NumPy, Pandas, GeoPandas. 

[DEBUG] Chatbot responding...
[DEBUG] Entering preprocess_input
[DEBUG] Preprocessed input: Farming

[DEBUG] Entering classify_input
[DEBUG] Classified as: interest_profiling
[DEBUG] Chatbot response: I'm not sure how to respond to that. Could you please ask about a specific Python math problem or concept you'd like help with?
Chatbot: I'm not sure how to respond to that. Could you please ask about a specific Python math problem or concept you'd like help with?
[DEBUG] Chatbot responding...
[DEBUG] Entering preprocess_input
[DEBUG] Preprocessed input: 

TypeError: 'Content' object is not subscriptable

# Tech Used
https://ai.google.dev/gemini-api/docs/function-calling/tutorial?lang=python

    Home
    Gemini API
    Docs

Was this helpful?
Function calling tutorial

Function calling makes it easier for you to get structured data outputs from generative models. You can then use these outputs to call other APIs and return the relevant response data to the model. In other words, function calling helps you connect generative models to external systems so that the generated content includes the most up-to-date and accurate information.

You can provide Gemini models with descriptions of functions. These are functions that you write in the language of your app (that is, they're not Google Cloud Functions). The model may ask you to call a function and send back the result to help the model handle your query.

If you haven't already, check out the Introduction to function calling to learn more. You can also try out this feature in Google Colab or view the example code in the Gemini API Cookbook repository.
Example API for lighting control

Imagine you have a basic lighting control system with an application programming interface (API) and you want to allow users to control the lights through simple text requests. You can use the Function Calling feature to interpret lighting change requests from users and translate them into API calls to set the lighting values. This hypothetical lighting control system lets you control the brightness of the light and it's color temperature, defined as two separate parameters:
Parameter 	Type 	Required 	Description
brightness 	number 	yes 	Light level from 0 to 100. Zero is off and 100 is full brightness.
colorTemperature 	string 	yes 	Color temperature of the light fixture which can be daylight, cool or warm.

For simplicity, this imaginary lighting system only has one light, so the user does not have to specify a room or location. Here is an example JSON request you could send to the lighting control API to change the light level to 50% using the daylight color temperature:

{
  "brightness": "50",
  "colorTemperature": "daylight"
}

This tutorial shows you how to set up a Function Call for the Gemini API to interpret users lighting requests and map them to API settings to control a light's brightness and color temperature values.
Before you begin: Set up your project and API key

Before calling the Gemini API, you need to set up your project and configure your API key.

Expand to view how to set up your project and API key

Define an API function

Create a function that makes an API request. This function should be defined within the code of your application, but could call services or APIs outside of your application. The Gemini API does not call this function directly, so you can control how and when this function is executed through your application code. For demonstration purposes, this tutorial defines a mock API function that just returns the requested lighting values:

def set_light_values(brightness, color_temp):
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {
        "brightness": brightness,
        "colorTemperature": color_temp
    }

When you create a function to be used in a function call by the model, you should include as much detail as possible in the function and parameter descriptions. The generative model uses this information to determine which function to select and how to provide values for the parameters in the function call.
Caution: For any production application, you should validate the data being passed to the API function from the model before executing the function.
Note: For programing languages other than Python, you must create create a separate function declaration for your API. See the other language programming tutorials more details.
Declare functions during model initialization

When you want to use function calling with a model, you must declare your functions when you initialize the model object. You declare functions by setting the model's tools parameter:

model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              tools=[set_light_values])

Generate a function call

Once you have initialized model with your function declarations, you can prompt the model with the defined function. You should use use function calling using chat prompting (sendMessage()), since function calling generally benefits from having the context of previous prompts and responses.

chat = model.start_chat()
response = chat.send_message('Dim the lights so the room feels cozy and warm.')
response.text

The Python SDK's ChatSession object simplifies managing chat sessions by handling the conversation history for you. You can use the enable_automatic_function_calling to have the SDK automatically

# Create a chat session that automatically makes suggested function calls
chat = model.start_chat(enable_automatic_function_calling=True)

Warning: Do not use this feature in production applications as there are no data input verification checks for automatic function calls.
Parallel function calling

In addition to basic function calling described above, you can also call multiple functions in a single turn. This section shows an example for how you can use parallel function calling.

Define the tools.

def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

Now call the model with an instruction that could use all of the specified tools.

# Set the model up with tools.
house_fns = [power_disco_ball, start_music, dim_lights]

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=house_fns)

# Call the API.
chat = model.start_chat()
response = chat.send_message("Turn this place into a party!")

# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

power_disco_ball(power=True)
start_music(energetic=True, loud=True, bpm=120.0)
dim_lights(brightness=0.3)

Each of the printed results reflects a single function call that the model has requested. To send the results back, include the responses in the same order as they were requested.

# Simulate the responses from the specified tools.
responses = {
    "power_disco_ball": True,
    "start_music": "Never gonna give you up.",
    "dim_lights": True,
}

# Build the response parts.
response_parts = [
    genai.protos.Part(function_response=genai.protos.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

response = chat.send_message(response_parts)
print(response.text)

Let's get this party started! I've turned on the disco ball, started playing some upbeat music, and dimmed the lights. 🎶✨  Get ready to dance! 🕺💃


# Code Execution

Enable code execution on the model

You can enable code execution on the model, as shown here:

import os
import google.generativeai as genai

genai.configure(api_key=os.environ['API_KEY'])

model = genai.GenerativeModel(
    model_name='gemini-1.5-pro',
    tools='code_execution')

response = model.generate_content((
    'What is the sum of the first 50 prime numbers? '
    'Generate and run code for the calculation, and make sure you get all 50.'))

print(response.text)

The output might look something like this:

```python
def is_prime(n):
  """Checks if a number is prime."""
  if n <= 1:
    return False
  for i in range(2, int(n**0.5) + 1):
    if n % i == 0:
      return False
  return True

def sum_of_primes(n):
  """Calculates the sum of the first n prime numbers."""
  primes = []
  i = 2
  while len(primes) < n:
    if is_prime(i):
      primes.append(i)
    i += 1
  return sum(primes)

# Calculate the sum of the first 50 prime numbers
sum_of_first_50_primes = sum_of_primes(50)

print(f"The sum of the first 50 prime numbers is: {sum_of_first_50_primes}")
```

**Explanation:**

1. **`is_prime(n)` Function:**
   - Takes an integer `n` as input.
   - Returns `False` for numbers less than or equal to 1 (not prime).
   - Iterates from 2 up to the square root of `n`. If `n` is divisible by any
     number in this range, it's not prime, and we return `False`.
   - If the loop completes without finding a divisor, the number is prime, and
     we return `True`.

2. **`sum_of_primes(n)` Function:**
   - Takes an integer `n` (number of primes desired) as input.
   - Initializes an empty list `primes` to store the prime numbers.
   - Starts a loop, iterating through numbers starting from 2.
   - For each number `i`, it checks if it's prime using the `is_prime()` function.
   - If `i` is prime, it's appended to the `primes` list.
   - The loop continues until the `primes` list has `n` prime numbers.
   - Finally, it calculates and returns the sum of all the prime numbers in the
     `primes` list.

3. **Main Part:**
   - Calls `sum_of_primes(50)` to get the sum of the first 50 prime numbers.
   - Prints the result.

**Output:**

```
The sum of the first 50 prime numbers is: 5117
```

Enable code execution on the request

Alternatively, you can enable code execution on the call to generate_content:

import os
import google.generativeai as genai

genai.configure(api_key=os.environ['API_KEY'])

model = genai.GenerativeModel(model_name='gemini-1.5-pro')

response = model.generate_content(
    ('What is the sum of the first 50 prime numbers? '
    'Generate and run code for the calculation, and make sure you get all 50.'),
    tools='code_execution')

print(response.text)

Use code execution in chat

You can also use code execution as part of a chat.

import os
import google.generativeai as genai

genai.configure(api_key=os.environ['API_KEY'])

model = genai.GenerativeModel(model_name='gemini-1.5-pro',
                              tools='code_execution')

chat = model.start_chat()

response = chat.send_message((
    'What is the sum of the first 50 prime numbers? '
    'Generate and run code for the calculation, and make sure you get all 50.'))

print(response.text)

Code execution versus function calling

Code execution and function calling are similar features:

    Code execution lets the model run code in the API backend in a fixed, isolated environment.
    Function calling lets you run the functions that the model requests, in whatever environment you want.

In general you should prefer to use code execution if it can handle your use case. Code execution is simpler to use (you just enable it) and resolves in a single GenerateContent request (thus incurring a single charge). Function calling takes an additional GenerateContent request to send back the output from each function call (thus incurring multiple charges).

For most cases, you should use function calling if you have your own functions that you want to run locally, and you should use code execution if you'd like the API to write and run Python code for you and return the result.

In [20]:
hellloo = model.generate_content("1+1")

print(hellloo.text)

2



In [51]:
import os
import google.generativeai as genai
import pathlib
import textwrap

from IPython.display import display
from IPython.display import Markdown

genai.configure(api_key="")


def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))


code_model = genai.GenerativeModel(model_name="gemini-1.5-pro", tools="code_execution")

response = code_model.generate_content(
    (
        "What is the sum of the first 50 prime numbers? "
        "Generate and run code for the calculation, and make sure you get all 50."
    ),
    tools="code_execution",
)

print(response.text)

I can write code to calculate that. First, I need a function to check if a number is prime. Then I can iterate through numbers, check if they're prime, and add them until I have 50 primes. 


``` python
def is_prime(n):
    """Checks if n is a prime number (greater than 1)"""
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True


primes = []
num = 2
while len(primes) < 50:
    if is_prime(num):
        primes.append(num)
    num += 1

print(sum(primes))

```
```
5117

```
The sum of the first 50 prime numbers is 5117. 



In [52]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True


# Set the model up with tools.
house_fns = [power_disco_ball, start_music, dim_lights]

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=house_fns)

# Call the API.
chat = model.start_chat()
response = chat.send_message("Turn this place into a party!")

In [53]:
# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

power_disco_ball(power=True)
start_music(loud=True, energetic=True, bpm=120.0)
dim_lights(brightness=0.5)


In [57]:
# Simulate the responses from the specified tools.
responses = {
    "power_disco_ball": True,
    "start_music": "Never gonna give you up.",
    "dim_lights": True,
}

# Build the response parts.
response_parts = [
    genai.protos.Part(
        function_response=genai.protos.FunctionResponse(
            name=fn, response={"result": val}
        )
    )
    for fn, val in responses.items()
]

response = chat.send_message(response_parts)
print(response.text)

AttributeError: whichOneof